# Data prepation with daily aggregation
The observation period is split into 8 periods and purchase history is aggregated by client and period and then flattened into a two-dimentional data set.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Client data

In [3]:
clients = pd.read_csv('clients2.csv',
                     index_col='client_id',
                     parse_dates=['first_issue_date', 'first_redeem_date'],
                     na_values={'gender': 'U'})\
    .drop(columns='client_id.1')
clients

,first_issue_date,first_redeem_date,age,gender
client_id,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,NaN
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,NaN
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,NaN
...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,NaN
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F


# Age

There are negative values and values above 100 (as high as 1852).

Calculate mean age using clients within reasonable age range:

In [4]:
mean_age = clients.loc[(clients.age < 90) & (clients.age > 10), 'age'].mean()
mean_age

46.37953291808248

In [5]:
clients.loc[(clients.age > 90) | (clients.age < 10), 'age'] = mean_age

# Gender

In [6]:
clients = pd.get_dummies(clients, columns=['gender'])
clients

,first_issue_date,first_redeem_date,age,gender_F,gender_M
client_id,,,,,
000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45.0,0,0
000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72.0,1,0
00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83.0,0,0
0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33.0,1,0
00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73.0,0,0
...,...,...,...,...,...
fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35.0,1,0
fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69.0,0,0
fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73.0,1,0


# Issue and redeem dates

## Clients that have not redeemed

In [7]:
clients['no_redeem'] = clients.first_redeem_date.isna().astype('int')

In [8]:
clients.isna().sum()

first_issue_date         0
first_redeem_date    17546
age                      0
gender_F                 0
gender_M                 0
no_redeem                0
dtype: int64

In [9]:
clients.loc[clients.no_redeem == 1, :]

,first_issue_date,first_redeem_date,age,gender_F,gender_M,no_redeem
client_id,,,,,,
00068fd5dc,2018-12-15 11:14:26,NaT,72.000000,1,0,1
0006fca4bf,2019-02-20 10:47:30,NaT,57.000000,0,0,1
0009e6bafa,2018-10-01 18:40:29,NaT,32.000000,0,0,1
000d599743,2018-08-29 09:08:39,NaT,46.379533,0,0,1
00140e5d34,2019-03-07 13:10:31,NaT,50.000000,1,0,1
...,...,...,...,...,...,...
ffe2d2bdbc,2018-02-16 12:06:15,NaT,82.000000,0,0,1
ffe4b6aa1a,2019-01-19 19:01:44,NaT,56.000000,0,0,1
fff2b6bf63,2018-10-18 19:01:34,NaT,46.000000,0,0,1


## Use the last redeem date to fill missing values

In [10]:
last_redeem = clients.first_redeem_date.max(); last_redeem

Timestamp('2019-11-20 01:14:10')

In [11]:
clients.fillna(last_redeem, inplace=True)

In [12]:
clients.isna().sum()

first_issue_date     0
first_redeem_date    0
age                  0
gender_F             0
gender_M             0
no_redeem            0
dtype: int64

In [13]:
clients['first_issue_date'] = (clients['first_issue_date'] - pd.Timestamp("1970-01-01")).dt.days
clients['first_redeem_date'] = (clients['first_redeem_date'] - pd.Timestamp("1970-01-01")).dt.days
clients

,first_issue_date,first_redeem_date,age,gender_F,gender_M,no_redeem
client_id,,,,,,
000012768d,17383,17535,45.0,0,0,0
000036f903,17266,17279,72.0,1,0,0
00010925a5,17736,17788,83.0,0,0,0
0001f552b0,17347,17771,33.0,1,0,0
00020e7b18,17497,17541,73.0,0,0,0
...,...,...,...,...,...,...
fffe0abb97,17497,17573,35.0,1,0,0
fffe0ed719,17424,17512,69.0,0,0,0
fffea1204c,17562,17602,73.0,1,0,0


### Scaling

In [14]:
contin_vars = ['first_issue_date', 'first_redeem_date', 'age']

In [15]:
scaler = StandardScaler()

In [16]:
scaled = scaler.fit_transform(clients[contin_vars])

In [17]:
clients[contin_vars] = pd.DataFrame(scaled, columns=contin_vars, index=clients.index)
clients

,first_issue_date,first_redeem_date,age,gender_F,gender_M,no_redeem
client_id,,,,,,
000012768d,-0.769796,-0.855610,-0.087012,0,0,0
000036f903,-1.344307,-1.861964,1.615538,1,0,0
00010925a5,0.963560,0.138951,2.309169,0,0,0
0001f552b0,-0.946568,0.072123,-0.843701,1,0,0
00020e7b18,-0.210015,-0.832023,1.678595,0,0,0
...,...,...,...,...,...,...
fffe0abb97,-0.210015,-0.706229,-0.717586,1,0,0
fffe0ed719,-0.568471,-0.946024,1.426365,0,0,0
fffea1204c,0.109158,-0.592228,1.678595,1,0,0


In [18]:
clients.to_csv('clients2_modified.csv')

# Purchase data aggregation

In [2]:
test_purch = pd.read_csv('test_purch.csv', parse_dates=['transaction_datetime'])

In [3]:
train_purch = pd.read_csv('train_purch.csv', parse_dates=['transaction_datetime'])
train_purch

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15998947,fffeca6d22,a0bb11a968,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,de437430e2,1.0,19.0,NaN
15998948,fffeca6d22,a0bb11a968,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,ad8fee4200,1.0,45.0,NaN
15998949,fffeca6d22,3b876a5532,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,ad8fee4200,2.0,94.0,NaN
15998950,fffeca6d22,3b876a5532,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,230dbb0c16,1.0,50.0,NaN


In [4]:
test_purch = test_purch\
    .groupby(['client_id','transaction_datetime'],
            as_index=False)\
    .agg({'regular_points_received': 'mean',
          'express_points_received': 'mean',
          'regular_points_spent': 'mean', 
          'express_points_spent': 'mean', 
          'purchase_sum': 'mean',
         'product_id': 'count',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
test_purch.rename(columns = {'product_id': 'n_products'}, inplace=True)
test_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,5,7.0,303.0,0.0
1,00010925a5,2018-12-01 09:17:18,0.8,0.0,0.0,0.0,132.0,3,2.0,132.0,0.0
2,00010925a5,2018-12-03 07:52:46,0.7,0.0,0.0,0.0,149.0,3,3.0,150.0,0.0
3,00010925a5,2018-12-06 09:59:19,1.7,0.0,0.0,0.0,349.0,6,7.0,349.0,0.0
4,00010925a5,2018-12-17 12:51:26,5.8,0.0,0.0,0.0,581.0,8,8.0,582.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1209945,fffff6ce77,2019-02-28 12:56:31,1.8,0.0,0.0,0.0,446.0,9,10.0,446.0,0.0
1209946,fffff6ce77,2019-03-05 15:07:03,0.3,0.0,0.0,0.0,247.0,1,1.0,247.0,0.0
1209947,fffff6ce77,2019-03-06 16:26:35,0.1,0.0,0.0,0.0,109.0,2,2.0,110.0,0.0
1209948,fffff6ce77,2019-03-06 16:53:10,0.2,0.0,0.0,0.0,245.0,1,1.0,245.0,0.0


In [5]:
train_purch = train_purch\
    .groupby(['client_id','transaction_datetime'],
            as_index=False)\
    .agg({'regular_points_received': 'mean',
          'express_points_received': 'mean',
          'regular_points_spent': 'mean', 
          'express_points_spent': 'mean', 
          'purchase_sum': 'mean',
         'product_id': 'count',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
train_purch.rename(columns = {'product_id': 'n_products'}, inplace=True)
train_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,19,21.0,1007.0,0.0
1,000012768d,2018-12-16 08:56:01,5.7,0.0,0.0,0.0,574.0,11,14.0,575.0,0.0
2,000012768d,2019-03-08 10:12:03,8.0,0.0,0.0,0.0,803.0,16,13.0,804.0,0.0
3,000012768d,2019-03-14 15:01:47,2.0,0.0,0.0,0.0,419.0,6,6.0,419.0,0.0
4,000036f903,2018-11-28 10:48:36,1.2,0.0,0.0,0.0,241.0,5,4.0,241.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2814986,fffeca6d22,2019-02-22 09:34:13,5.0,0.0,0.0,0.0,110.0,3,3.0,111.0,0.0
2814987,fffeca6d22,2019-02-22 09:34:53,6.0,0.0,0.0,0.0,138.0,1,2.0,139.0,0.0
2814988,fffeca6d22,2019-02-28 10:25:24,0.6,0.0,0.0,0.0,128.0,2,3.0,128.0,0.0
2814989,fffeca6d22,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,3,3.0,100.0,0.0


In [6]:
train_purch.transaction_datetime.min()

Timestamp('2018-11-21 21:02:33')

In [7]:
train_purch.transaction_datetime.max()

Timestamp('2019-03-18 23:19:28')

In [8]:
test_purch.transaction_datetime.min()

Timestamp('2018-11-21 21:02:51')

In [9]:
test_purch.transaction_datetime.max()

Timestamp('2019-03-18 22:08:09')

In [10]:
start = min(
    train_purch.transaction_datetime.min(),
    test_purch.transaction_datetime.min()
)
start

Timestamp('2018-11-21 21:02:33')

In [11]:
finish = max(
    train_purch.transaction_datetime.max(),
    test_purch.transaction_datetime.max()
)
finish

Timestamp('2019-03-18 23:19:28')

Number of intervals:

In [12]:
n_intervals = 16

In [13]:
bins = [start + i * (finish - start) / n_intervals for i in range(0, n_intervals + 1)]
bins

[Timestamp('2018-11-21 21:02:33'),
 Timestamp('2018-11-29 04:41:06.437500'),
 Timestamp('2018-12-06 12:19:39.875000'),
 Timestamp('2018-12-13 19:58:13.312500'),
 Timestamp('2018-12-21 03:36:46.750000'),
 Timestamp('2018-12-28 11:15:20.187500'),
 Timestamp('2019-01-04 18:53:53.625000'),
 Timestamp('2019-01-12 02:32:27.062500'),
 Timestamp('2019-01-19 10:11:00.500000'),
 Timestamp('2019-01-26 17:49:33.937500'),
 Timestamp('2019-02-03 01:28:07.375000'),
 Timestamp('2019-02-10 09:06:40.812500'),
 Timestamp('2019-02-17 16:45:14.250000'),
 Timestamp('2019-02-25 00:23:47.687500'),
 Timestamp('2019-03-04 08:02:21.125000'),
 Timestamp('2019-03-11 15:40:54.562500'),
 Timestamp('2019-03-18 23:19:28')]

In [14]:
test_purch['interval'] = pd.cut(test_purch.transaction_datetime, bins, labels=False)
test_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red,interval
0,00010925a5,2018-11-25 10:56:18,1.5,0.0,0.0,0.0,303.0,5,7.0,303.0,0.0,0
1,00010925a5,2018-12-01 09:17:18,0.8,0.0,0.0,0.0,132.0,3,2.0,132.0,0.0,1
2,00010925a5,2018-12-03 07:52:46,0.7,0.0,0.0,0.0,149.0,3,3.0,150.0,0.0,1
3,00010925a5,2018-12-06 09:59:19,1.7,0.0,0.0,0.0,349.0,6,7.0,349.0,0.0,1
4,00010925a5,2018-12-17 12:51:26,5.8,0.0,0.0,0.0,581.0,8,8.0,582.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1209945,fffff6ce77,2019-02-28 12:56:31,1.8,0.0,0.0,0.0,446.0,9,10.0,446.0,0.0,13
1209946,fffff6ce77,2019-03-05 15:07:03,0.3,0.0,0.0,0.0,247.0,1,1.0,247.0,0.0,14
1209947,fffff6ce77,2019-03-06 16:26:35,0.1,0.0,0.0,0.0,109.0,2,2.0,110.0,0.0,14
1209948,fffff6ce77,2019-03-06 16:53:10,0.2,0.0,0.0,0.0,245.0,1,1.0,245.0,0.0,14


In [15]:
train_purch['interval'] = pd.cut(train_purch.transaction_datetime, bins, labels=False)
train_purch

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red,interval
0,000012768d,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,19,21.0,1007.0,0.0,1.0
1,000012768d,2018-12-16 08:56:01,5.7,0.0,0.0,0.0,574.0,11,14.0,575.0,0.0,3.0
2,000012768d,2019-03-08 10:12:03,8.0,0.0,0.0,0.0,803.0,16,13.0,804.0,0.0,14.0
3,000012768d,2019-03-14 15:01:47,2.0,0.0,0.0,0.0,419.0,6,6.0,419.0,0.0,15.0
4,000036f903,2018-11-28 10:48:36,1.2,0.0,0.0,0.0,241.0,5,4.0,241.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2814986,fffeca6d22,2019-02-22 09:34:13,5.0,0.0,0.0,0.0,110.0,3,3.0,111.0,0.0,12.0
2814987,fffeca6d22,2019-02-22 09:34:53,6.0,0.0,0.0,0.0,138.0,1,2.0,139.0,0.0,12.0
2814988,fffeca6d22,2019-02-28 10:25:24,0.6,0.0,0.0,0.0,128.0,2,3.0,128.0,0.0,13.0
2814989,fffeca6d22,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,3,3.0,100.0,0.0,14.0


In [16]:
test_purch.isna().sum()

client_id                  0
transaction_datetime       0
regular_points_received    0
express_points_received    0
regular_points_spent       0
express_points_spent       0
purchase_sum               0
n_products                 0
product_quantity           0
trn_sum_from_iss           0
trn_sum_from_red           0
interval                   0
dtype: int64

In [17]:
train_purch.isna().sum()

client_id                  0
transaction_datetime       0
regular_points_received    0
express_points_received    0
regular_points_spent       0
express_points_spent       0
purchase_sum               0
n_products                 0
product_quantity           0
trn_sum_from_iss           0
trn_sum_from_red           0
interval                   1
dtype: int64

In [18]:
train_purch.loc[train_purch.interval.isna()]

,client_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red,interval
434367,27df8bc8bb,2018-11-21 21:02:33,1.5,0.0,0.0,0.0,224.53,5,8.0,226.0,0.0,NaN


In [19]:
train_purch.fillna(value=0, inplace=True)

In [20]:
train_purch.isna().sum()

client_id                  0
transaction_datetime       0
regular_points_received    0
express_points_received    0
regular_points_spent       0
express_points_spent       0
purchase_sum               0
n_products                 0
product_quantity           0
trn_sum_from_iss           0
trn_sum_from_red           0
interval                   0
dtype: int64

In [21]:
train_purch.dtypes

client_id                          object
transaction_datetime       datetime64[ns]
regular_points_received           float64
express_points_received           float64
regular_points_spent              float64
express_points_spent              float64
purchase_sum                      float64
n_products                          int64
product_quantity                  float64
trn_sum_from_iss                  float64
trn_sum_from_red                  float64
interval                          float64
dtype: object

In [22]:
train_purch.interval = train_purch.interval.astype('int')

In [23]:
train_purch.dtypes

client_id                          object
transaction_datetime       datetime64[ns]
regular_points_received           float64
express_points_received           float64
regular_points_spent              float64
express_points_spent              float64
purchase_sum                      float64
n_products                          int64
product_quantity                  float64
trn_sum_from_iss                  float64
trn_sum_from_red                  float64
interval                            int64
dtype: object

In [24]:
test_purch = test_purch\
    .groupby(['client_id','interval'])\
    .agg({'regular_points_received': 'sum',
          'express_points_received': 'sum',
          'regular_points_spent': 'sum', 
          'express_points_spent': 'sum', 
          'purchase_sum': 'sum',
         'n_products': 'sum',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
test_purch

regular_points_received  express_points_received  \
client_id  interval                                                     
00010925a5 0                             1.5                      0.0   
           1                             3.2                      0.0   
           3                             6.7                      0.0   
           4                             2.1                      0.0   
           9                             2.3                      0.0   
...                                      ...                      ...   
fffff6ce77 11                            1.8                      0.0   
           12                           16.5                      0.0   
           13                            3.5                      0.0   
           14                            0.6                      0.0   
           15                            6.7                      0.0   

                     regular_points_spent  express_points_spent  purchase_sum  \
client_id  interval                                                             
00010925a5 0                          0.0                   0.0         303.0   
           1                          0.0                   0.0         630.0   
           3                          0.0                   0.0         764.0   
           4                        -17.0                   0.0         438.0   
           9                          0.0                   0.0         463.0   
...                                   ...                   ...           ...   
fffff6ce77 11                         0.0                   0.0         558.0   
           12                         0.0                   0.0        2286.0   
           13                         0.0                   0.0         870.0   
           14                         0.0                   0.0         601.0   
           15                         0.0                   0.0         676.0   

                     n_products  product_quantity  trn_sum_from_iss  \
client_id  interval                                                   
00010925a5 0                  5               7.0             303.0   
           1                 12              12.0             631.0   
           3                 11              11.0             766.0   
           4                  5               4.0             420.0   
           9                  5               5.0             463.0   
...                         ...               ...               ...   
fffff6ce77 11                11              11.0             560.0   
           12                17              25.0            2289.0   
           13                14              15.0             871.0   
           14                 4               4.0             602.0   
           15                 9               9.0             676.0   

                     trn_sum_from_red  
client_id  interval                    
00010925a5 0                      0.0  
           1                      0.0  
           3                      0.0  
           4                    438.0  
           9                      0.0  
...                               ...  
fffff6ce77 11                     0.0  
           12                     0.0  
           13                     0.0  
           14                     0.0  
           15                     0.0  

[557755 rows x 9 columns]

In [25]:
train_purch = train_purch\
    .groupby(['client_id','interval'])\
    .agg({'regular_points_received': 'sum',
          'express_points_received': 'sum',
          'regular_points_spent': 'sum', 
          'express_points_spent': 'sum', 
          'purchase_sum': 'sum',
         'n_products': 'sum',
         'product_quantity': 'sum',
         'trn_sum_from_iss': 'sum',
         'trn_sum_from_red': 'sum'})
train_purch

regular_points_received  express_points_received  \
client_id  interval                                                     
000012768d 1                            10.0                      0.0   
           3                             5.7                      0.0   
           14                            8.0                      0.0   
           15                            2.0                      0.0   
000036f903 0                             1.2                      0.0   
...                                      ...                      ...   
fffeca6d22 10                            0.4                      0.0   
           12                           32.7                      0.0   
           13                            0.6                      0.0   
           14                            0.4                      0.0   
           15                            0.8                      0.0   

                     regular_points_spent  express_points_spent  purchase_sum  \
client_id  interval                                                             
000012768d 1                          0.0                   0.0       1007.00   
           3                          0.0                   0.0        574.00   
           14                         0.0                   0.0        803.00   
           15                         0.0                   0.0        419.00   
000036f903 0                          0.0                   0.0        241.00   
...                                   ...                   ...           ...   
fffeca6d22 10                         0.0                   0.0         97.00   
           12                         0.0                   0.0        859.86   
           13                         0.0                   0.0        128.00   
           14                         0.0                   0.0         99.00   
           15                         0.0                   0.0        165.00   

                     n_products  product_quantity  trn_sum_from_iss  \
client_id  interval                                                   
000012768d 1                 19              21.0            1007.0   
           3                 11              14.0             575.0   
           14                16              13.0             804.0   
           15                 6               6.0             419.0   
000036f903 0                  5               4.0             241.0   
...                         ...               ...               ...   
fffeca6d22 10                 2               2.0              98.0   
           12                22              25.0             864.0   
           13                 2               3.0             128.0   
           14                 3               3.0             100.0   
           15                 3               4.0             166.0   

                     trn_sum_from_red  
client_id  interval                    
000012768d 1                      0.0  
           3                      0.0  
           14                     0.0  
           15                     0.0  
000036f903 0                      0.0  
...                               ...  
fffeca6d22 10                     0.0  
           12                     0.0  
           13                     0.0  
           14                     0.0  
           15                     0.0  

[1302946 rows x 9 columns]

In [26]:
scaler_2 = StandardScaler().fit(train_purch)

In [27]:
scaled_train = scaler_2.transform(train_purch)

In [28]:
train_purch = pd.DataFrame(scaled_train,
                           columns=train_purch.columns,
                           index=train_purch.index)
train_purch.reset_index(inplace=True)
train_purch

,client_id,interval,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,1,0.112551,-0.03349,0.204198,0.145555,0.084434,0.616687,0.370313,0.108419,-0.23752
1,000012768d,3,-0.183198,-0.03349,0.204198,0.145555,-0.348431,-0.117362,-0.084432,-0.333902,-0.23752
2,000012768d,14,-0.025006,-0.03349,0.204198,0.145555,-0.119502,0.341419,-0.149396,-0.099431,-0.23752
3,000012768d,15,-0.437679,-0.03349,0.204198,0.145555,-0.503382,-0.576142,-0.604142,-0.493630,-0.23752
4,000036f903,0,-0.492703,-0.03349,0.204198,0.145555,-0.681327,-0.667898,-0.734069,-0.675882,-0.23752
...,...,...,...,...,...,...,...,...,...,...,...
1302941,fffeca6d22,10,-0.547726,-0.03349,0.204198,0.145555,-0.825282,-0.943166,-0.863996,-0.822299,-0.23752
1302942,fffeca6d22,12,1.673831,-0.03349,0.204198,0.145555,-0.062660,0.891955,0.630168,-0.037998,-0.23752
1302943,fffeca6d22,13,-0.533970,-0.03349,0.204198,0.145555,-0.794291,-0.943166,-0.799033,-0.791582,-0.23752
1302944,fffeca6d22,14,-0.547726,-0.03349,0.204198,0.145555,-0.823282,-0.851410,-0.799033,-0.820251,-0.23752


In [29]:
scaled_test = scaler_2.transform(test_purch)

In [30]:
test_purch = pd.DataFrame(scaled_test,
                           columns=test_purch.columns,
                           index=test_purch.index)
test_purch.reset_index(inplace=True)
test_purch

,client_id,interval,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,n_products,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,00010925a5,0,-0.472069,-0.03349,0.204198,0.145555,-0.619346,-0.667898,-0.539178,-0.612401,-0.237520
1,00010925a5,1,-0.355145,-0.03349,0.204198,0.145555,-0.292448,-0.025605,-0.214360,-0.276564,-0.237520
2,00010925a5,3,-0.114419,-0.03349,0.204198,0.145555,-0.158490,-0.117362,-0.279323,-0.138339,-0.237520
3,00010925a5,4,-0.430802,-0.03349,-0.237475,0.145555,-0.484388,-0.667898,-0.734069,-0.492606,1.437596
4,00010925a5,9,-0.417046,-0.03349,0.204198,0.145555,-0.459396,-0.667898,-0.669105,-0.448578,-0.237520
...,...,...,...,...,...,...,...,...,...,...,...
557750,fffff6ce77,11,-0.451435,-0.03349,0.204198,0.145555,-0.364426,-0.117362,-0.279323,-0.349261,-0.237520
557751,fffff6ce77,12,0.559614,-0.03349,0.204198,0.145555,1.363034,0.433175,0.630168,1.421048,-0.237520
557752,fffff6ce77,13,-0.334511,-0.03349,0.204198,0.145555,-0.052523,0.157907,-0.019469,-0.030830,-0.237520
557753,fffff6ce77,14,-0.533970,-0.03349,0.204198,0.145555,-0.321439,-0.759654,-0.734069,-0.306257,-0.237520


In [31]:
test_purch = test_purch.pivot(index='client_id', columns='interval')
test_purch.fillna(0, inplace=True)
test_purch

regular_points_received                                          \
interval                        0         1         2         3         4    
client_id                                                                    
00010925a5               -0.472069 -0.355145  0.000000 -0.114419 -0.430802   
00035a21d9                0.000000  0.000000  0.000000  0.000000  0.000000   
00038f9200                0.000000  0.000000 -0.430802  0.483957 -0.313878   
0004315e57               -0.203831  0.000000  0.000000  0.000000  0.435812   
0006fca4bf                0.000000  0.000000  0.000000  0.000000  0.000000   
...                            ...       ...       ...       ...       ...   
fff9aa288c                0.000000  0.112551  0.000000  0.000000  0.000000   
fff9f772f6                0.000000  0.000000  0.000000  0.380789  0.000000   
fffe0ed719               -0.458313  0.000000  0.000000 -0.293244 -0.527092   
fffea1204c                0.000000 -0.417046 -0.520214  0.000000 -0.527092   
fffff6ce77                0.000000  0.745317  1.041066  0.000000  0.518346   

                                                              ...  \
interval          5         6         7         8         9   ...   
client_id                                                     ...   
00010925a5  0.000000  0.000000  0.000000  0.000000 -0.417046  ...   
00035a21d9  0.000000  0.000000  0.000000  0.000000  0.000000  ...   
00038f9200 -0.025006 -0.547726 -0.279488 -0.410168 -0.396412  ...   
0004315e57 -0.417046  0.000000  0.000000  0.000000 -0.121297  ...   
0006fca4bf  0.000000  0.000000  0.000000  0.000000  0.000000  ...   
...              ...       ...       ...       ...       ...  ...   
fff9aa288c  0.000000  0.000000 -0.472069  0.944775  0.000000  ...   
fff9f772f6  0.000000  0.000000 -0.045640  0.000000  0.000000  ...   
fffe0ed719 -0.533970 -0.478947  0.000000  0.000000 -0.520214  ...   
fffea1204c  0.000000 -0.499580 -0.478947 -0.554604 -0.403290  ...   
fffff6ce77  0.174452  1.419349  1.144234 -0.245099  3.241989  ...   

           trn_sum_from_red                                                  \
interval                 6         7        8         9         10       11   
client_id                                                                     
00010925a5          0.00000  0.000000  0.00000 -0.237520 -0.237520  0.00000   
00035a21d9          0.00000  0.000000  0.00000  0.000000 -0.237520 -0.23752   
00038f9200         -0.23752  1.070448 -0.23752 -0.237520 -0.237520 -0.23752   
0004315e57          0.00000  0.000000  0.00000  2.493149 -0.237520 -0.23752   
0006fca4bf          0.00000  0.000000  0.00000  0.000000  0.000000  0.00000   
...                     ...       ...      ...       ...       ...      ...   
fff9aa288c          0.00000 -0.237520 -0.23752  0.000000 -0.237520 -0.23752   
fff9f772f6          0.00000 -0.237520  0.00000  0.000000  0.000000  0.00000   
fffe0ed719         -0.23752  0.000000  0.00000 -0.237520  0.000000 -0.23752   
fffea1204c         -0.23752 -0.237520 -0.23752 -0.237520 -0.237520 -0.23752   
fffff6ce77         -0.23752 -0.237520 -0.23752 -0.237520  3.797292 -0.23752   

                                                  
interval         12        13        14       15  
client_id                                         
00010925a5 -0.23752 -0.237520 -0.237520  0.00000  
00035a21d9  0.00000  2.772335  0.000000  0.00000  
00038f9200 -0.23752 -0.237520 -0.237520 -0.23752  
0004315e57  0.00000  2.489325 -0.237520  0.00000  
0006fca4bf -0.23752  0.000000 -0.237520  0.00000  
...             ...       ...       ...      ...  
fff9aa288c  0.00000  0.000000  0.496778  0.00000  
fff9f772f6  0.00000 -0.237520  0.000000 -0.23752  
fffe0ed719 -0.23752 -0.237520 -0.237520 -0.23752  
fffea1204c -0.23752  0.661230  0.000000 -0.23752  
fffff6ce77 -0.23752 -0.237520 -0.237520 -0.23752  

[60012 rows x 144 columns]

In [32]:
train_purch = train_purch.pivot(index='client_id', columns='interval')
train_purch.fillna(0, inplace=True)
train_purch

regular_points_received                                          \
interval                        0         1         2         3         4    
client_id                                                                    
000012768d                0.000000  0.112551  0.000000 -0.183198  0.000000   
000036f903               -0.492703 -0.313878 -0.437679 -0.410168  0.435812   
0001f552b0                0.050650 -0.066274  0.000000  0.000000  0.000000   
00020e7b18                2.492299  1.818267  0.000000  0.000000  2.485422   
000220a0a7                0.000000  0.000000  0.456446  0.000000  2.678002   
...                            ...       ...       ...       ...       ...   
fffd5cd0c6                0.000000 -0.492703 -0.465191 -0.327633 -0.547726   
fffd63dfe3                0.000000  0.000000  0.000000  0.000000  0.000000   
fffd8c9d7d                0.000000 -0.540848  0.000000  0.000000  0.000000   
fffe0abb97                0.000000 -0.148808  0.000000  0.000000  0.000000   
fffeca6d22                0.000000  0.023139 -0.423924  0.000000  0.000000   

                                                              ...  \
interval          5         6         7         8         9   ...   
client_id                                                     ...   
000012768d  0.000000  0.000000  0.000000  0.000000  0.000000  ...   
000036f903  0.000000 -0.403290 -0.465191 -0.499580 -0.355145  ...   
0001f552b0  0.229475  0.000000  0.000000 -0.540848 -0.506458  ...   
00020e7b18  0.000000  2.423521  0.000000 -0.320755  0.146941  ...   
000220a0a7  0.000000  0.000000  0.000000  0.256987 -0.423924  ...   
...              ...       ...       ...       ...       ...  ...   
fffd5cd0c6  0.000000 -0.547726 -0.341389  0.133185 -0.472069  ...   
fffd63dfe3  0.000000  0.000000 -0.492703 -0.403290 -0.492703  ...   
fffd8c9d7d  0.000000  0.000000  0.000000 -0.169442  0.000000  ...   
fffe0abb97 -0.499580  0.000000  0.000000  0.000000 -0.100663  ...   
fffeca6d22  0.000000  0.000000 -0.547726 -0.540848 -0.527092  ...   

           trn_sum_from_red                                                  \
interval                 6        7         8         9        10        11   
client_id                                                                     
000012768d          0.00000  0.00000  0.000000  0.000000  0.00000  0.000000   
000036f903         -0.23752 -0.23752 -0.237520 -0.237520 -0.23752 -0.237520   
0001f552b0          0.00000  0.00000 -0.237520 -0.237520  0.00000 -0.237520   
00020e7b18         -0.23752  0.00000  3.089766  3.854659 -0.23752  6.765078   
000220a0a7          0.00000  0.00000 -0.237520 -0.237520 -0.23752 -0.237520   
...                     ...      ...       ...       ...      ...       ...   
fffd5cd0c6         -0.23752 -0.23752 -0.237520 -0.237520  0.00000  0.000000   
fffd63dfe3          0.00000 -0.23752 -0.237520 -0.237520  0.00000  0.000000   
fffd8c9d7d          0.00000  0.00000 -0.237520  0.000000  0.00000  0.000000   
fffe0abb97          0.00000  0.00000  0.000000  0.068437 -0.23752  0.000000   
fffeca6d22          0.00000 -0.23752 -0.237520 -0.237520 -0.23752  0.000000   

                                                  
interval         12       13        14        15  
client_id                                         
000012768d  0.00000  0.00000 -0.237520 -0.237520  
000036f903 -0.23752 -0.23752 -0.237520 -0.237520  
0001f552b0 -0.23752 -0.23752 -0.237520 -0.237520  
00020e7b18  7.50320 -0.23752  0.000000  0.106682  
000220a0a7 -0.23752 -0.23752 -0.237520 -0.237520  
...             ...      ...       ...       ...  
fffd5cd0c6 -0.23752 -0.23752  0.000000 -0.237520  
fffd63dfe3  0.00000  0.00000  0.000000 -0.237520  
fffd8c9d7d -0.23752 -0.23752 -0.237520  0.000000  
fffe0abb97 -0.23752  0.00000 -0.073068 -0.061594  
fffeca6d22 -0.23752 -0.23752 -0.237520 -0.237520  

[140027 rows x 144 columns]

In [33]:
columns = train_purch.columns.to_flat_index().to_list()
columns = [col[0] + '_' + str(col[1]) for col in columns]

In [34]:
train_purch.columns = columns
train_purch

,regular_points_received_0,regular_points_received_1,regular_points_received_2,regular_points_received_3,regular_points_received_4,regular_points_received_5,regular_points_received_6,regular_points_received_7,regular_points_received_8,regular_points_received_9,...,trn_sum_from_red_6,trn_sum_from_red_7,trn_sum_from_red_8,trn_sum_from_red_9,trn_sum_from_red_10,trn_sum_from_red_11,trn_sum_from_red_12,trn_sum_from_red_13,trn_sum_from_red_14,trn_sum_from_red_15
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,0.000000,0.112551,0.000000,-0.183198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,-0.237520,-0.237520
000036f903,-0.492703,-0.313878,-0.437679,-0.410168,0.435812,0.000000,-0.403290,-0.465191,-0.499580,-0.355145,...,-0.23752,-0.23752,-0.237520,-0.237520,-0.23752,-0.237520,-0.23752,-0.23752,-0.237520,-0.237520
0001f552b0,0.050650,-0.066274,0.000000,0.000000,0.000000,0.229475,0.000000,0.000000,-0.540848,-0.506458,...,0.00000,0.00000,-0.237520,-0.237520,0.00000,-0.237520,-0.23752,-0.23752,-0.237520,-0.237520
00020e7b18,2.492299,1.818267,0.000000,0.000000,2.485422,0.000000,2.423521,0.000000,-0.320755,0.146941,...,-0.23752,0.00000,3.089766,3.854659,-0.23752,6.765078,7.50320,-0.23752,0.000000,0.106682
000220a0a7,0.000000,0.000000,0.456446,0.000000,2.678002,0.000000,0.000000,0.000000,0.256987,-0.423924,...,0.00000,0.00000,-0.237520,-0.237520,-0.23752,-0.237520,-0.23752,-0.23752,-0.237520,-0.237520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffd5cd0c6,0.000000,-0.492703,-0.465191,-0.327633,-0.547726,0.000000,-0.547726,-0.341389,0.133185,-0.472069,...,-0.23752,-0.23752,-0.237520,-0.237520,0.00000,0.000000,-0.23752,-0.23752,0.000000,-0.237520
fffd63dfe3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.492703,-0.403290,-0.492703,...,0.00000,-0.23752,-0.237520,-0.237520,0.00000,0.000000,0.00000,0.00000,0.000000,-0.237520
fffd8c9d7d,0.000000,-0.540848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.169442,0.000000,...,0.00000,0.00000,-0.237520,0.000000,0.00000,0.000000,-0.23752,-0.23752,-0.237520,0.000000


In [35]:
test_purch.columns = columns
test_purch

,regular_points_received_0,regular_points_received_1,regular_points_received_2,regular_points_received_3,regular_points_received_4,regular_points_received_5,regular_points_received_6,regular_points_received_7,regular_points_received_8,regular_points_received_9,...,trn_sum_from_red_6,trn_sum_from_red_7,trn_sum_from_red_8,trn_sum_from_red_9,trn_sum_from_red_10,trn_sum_from_red_11,trn_sum_from_red_12,trn_sum_from_red_13,trn_sum_from_red_14,trn_sum_from_red_15
client_id,,,,,,,,,,,,,,,,,,,,,
00010925a5,-0.472069,-0.355145,0.000000,-0.114419,-0.430802,0.000000,0.000000,0.000000,0.000000,-0.417046,...,0.00000,0.000000,0.00000,-0.237520,-0.237520,0.00000,-0.23752,-0.237520,-0.237520,0.00000
00035a21d9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.000000,-0.237520,-0.23752,0.00000,2.772335,0.000000,0.00000
00038f9200,0.000000,0.000000,-0.430802,0.483957,-0.313878,-0.025006,-0.547726,-0.279488,-0.410168,-0.396412,...,-0.23752,1.070448,-0.23752,-0.237520,-0.237520,-0.23752,-0.23752,-0.237520,-0.237520,-0.23752
0004315e57,-0.203831,0.000000,0.000000,0.000000,0.435812,-0.417046,0.000000,0.000000,0.000000,-0.121297,...,0.00000,0.000000,0.00000,2.493149,-0.237520,-0.23752,0.00000,2.489325,-0.237520,0.00000
0006fca4bf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,-0.23752,0.000000,-0.237520,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff9aa288c,0.000000,0.112551,0.000000,0.000000,0.000000,0.000000,0.000000,-0.472069,0.944775,0.000000,...,0.00000,-0.237520,-0.23752,0.000000,-0.237520,-0.23752,0.00000,0.000000,0.496778,0.00000
fff9f772f6,0.000000,0.000000,0.000000,0.380789,0.000000,0.000000,0.000000,-0.045640,0.000000,0.000000,...,0.00000,-0.237520,0.00000,0.000000,0.000000,0.00000,0.00000,-0.237520,0.000000,-0.23752
fffe0ed719,-0.458313,0.000000,0.000000,-0.293244,-0.527092,-0.533970,-0.478947,0.000000,0.000000,-0.520214,...,-0.23752,0.000000,0.00000,-0.237520,0.000000,-0.23752,-0.23752,-0.237520,-0.237520,-0.23752


In [36]:
test_purch.to_csv('test_purch_16_periods.csv')

In [37]:
train_purch.to_csv('train_purch_16_periods.csv')